In [ ]:
from prophet import Prophet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error,root_mean_squared_error
import warnings
warnings.filterwarnings('ignore')


In [1]:
import matplotlib.pyplot as plt

In [ ]:
from pycaret.time_series import *

In [ ]:
df = pd.read_csv(r'..\..\data\processed\Merch_ML.csv',index_col='Order_ID')
df.head()

In [ ]:
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

In [ ]:
df.drop(df[(df['Order_Date'].dt.month == 11) & (df['Order_Date'].dt.year == 2024)].index,axis=0,inplace=True)

In [ ]:
# Asegurar que las fechas sean de tipo datetime
df['Order_Date'] = pd.to_datetime(df['Order_Date'])

# Agrupar por mes y sumar las ventas
df_monthly = df.groupby(df['Order_Date'].dt.to_period('M'))['Total_Sales'].sum().reset_index()

# Convertir 'Order_Date' de Period a Timestamp
df_monthly['Order_Date'] = df_monthly['Order_Date'].dt.to_timestamp()

# Verificar los datos después de la agrupación
print(df_monthly.head())


In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df_monthly['Order_Date'], df_monthly['Total_Sales'], marker='o', linestyle='-')
plt.xlabel('Fecha')
plt.ylabel('Ventas Mensuales')
plt.title('Ventas Mensuales a lo Largo del Tiempo')
plt.grid()
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(df_monthly['Total_Sales'])
print(f"p-value de Dickey-Fuller: {result[1]:.5f}")

# Interpretación:
if result[1] < 0.05:
    print("✅ La serie es estacionaria (p-value < 0.05)")
else:
    print("❌ La serie NO es estacionaria (p-value >= 0.05), necesitamos diferenciación.")


In [ ]:
print(f"Número de observaciones en df_monthly: {len(df_monthly)}")


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plt.figure(figsize=(12,5))

plt.subplot(121)
plot_acf(df_monthly['Total_Sales'].dropna(), lags=10)  # Ver ACF para determinar `q`
plt.title('ACF (Autocorrelación)')

plt.subplot(122)
plot_pacf(df_monthly['Total_Sales'].dropna(), lags=10)  # Ver PACF para determinar `p`
plt.title('PACF (Autocorrelación Parcial)')

plt.show()


In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Definir valores de p, d y q (ajustar según ACF/PACF)
p, d, q = 1, 0, 1  # Ajusta estos valores según tu análisis

# Entrenar ARIMA
model = ARIMA(df_monthly['Total_Sales'], order=(p, d, q))
model_fit = model.fit()

# Ver resumen del modelo
print(model_fit.summary())


In [ ]:
# Hacer predicciones para los próximos 3 meses
forecast = model_fit.forecast(steps=3)

# Crear un DataFrame con las fechas futuras
future_dates = pd.date_range(start=df_monthly['Order_Date'].max(), periods=4, freq='M')[1:]
forecast_df = pd.DataFrame({'Fecha': future_dates, 'Predicción': forecast})

print("\nPredicciones para los próximos 3 meses:")
print(forecast_df)


In [ ]:
df['Total_Sales'] = np.log1p(df['Total_Sales'])